### In-vitro data

In [ ]:
from pathlib import Path
import spikeinterface.full as si

%matplotlib widget

In [ ]:
base_data_folder = Path("/project/dlb/Dataset/SpikeImterface")
working_folder = Path("/localscratch")
recording_stem = "human_ngn2_100K_c_55D"
recording_name = f"{recording_stem}.brw"

results_folder = working_folder / recording_stem

In [ ]:
recording = si.read_biocam(base_data_folder / recording_name)

In [ ]:
recording

In [ ]:
recording = si.unsigned_to_signed(recording, bit_depth=12)

In [ ]:
recording

In [ ]:
recording_processed = si.bandpass_filter(recording, freq_max=5000)
recording_processed = si.common_reference(recording_processed)

In [ ]:
from spikeinterface.sortingcomponents.peak_detection import detect_peaks
from spikeinterface.sortingcomponents.peak_localization import localize_peaks

In [ ]:
si.set_global_job_kwargs(n_jobs=0.8, chunk_duration="0.5s")

In [ ]:
peaks = detect_peaks(recording_processed)

In [ ]:
si.plot_peak_activity(recording, peaks, color_range=[0, 10])

In [ ]:
sorting_HS = si.run_sorter(sorter_name="herdingspikes", recording=recording_processed, 
                           folder=working_folder / recording_name / "sorter_HS",
                           verbose=True, remove_existing_folder=True)

In [ ]:
sorting_HS

In [ ]:
analyzer = si.create_sorting_analyzer(sorting_HS, recording_processed)
extension_list = ["random_spikes", "templates", "unit_locations", "correlograms", "template_similarity", "spike_amplitudes"]
analyzer.compute(extension_list)

In [ ]:
si.plot_unit_templates(analyzer, backend="ipywidgets")

In [ ]:
si.plot_rasters(analyzer)

In [ ]:
potential_merges = si.compute_merge_unit_groups(analyzer, preset="x_contaminations")

In [ ]:
print(len(potential_merges))

In [ ]:
si.plot_potential_merges(analyzer, potential_merges, backend="ipywidgets")